In [5]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
%matplotlib inline

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

In [2]:
colnames=['date', 'time', 'page', 'user', 'cohort', 'ip']
df = pd.read_csv('curriculum-access.txt', engine='python',
                 header=None, index_col=False, names=colnames,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
                 na_values='"-"')

In [3]:
# combine date + time columns, then make it a datetime object, then drop the old columns and set datetime to index
df['datetime'] = pd.to_datetime(df.date + ' ' + df.time)
df = df.drop(columns=['date', 'time']).set_index('datetime')
# drop null page value
df = df.dropna(subset=['page'])
df.head()

,page,user,cohort,ip
datetime,,,,
2018-01-26 09:55:03,/,1,8.0,98.106.20.62
2018-01-26 09:56:02,java-ii,1,8.0,98.106.20.62
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,98.106.20.62
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,98.106.20.62
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,98.106.20.62


In [4]:
# fill nulls in Ada cohort
df[(df.user >= 349) & (df.user <= 369)] = df[(df.user >= 349) & (df.user <= 369)].fillna(30.0)
df[df.user == 429] = df[df.user == 429].fillna(30.0)
df[df.user == 48] = df[df.user == 48].fillna(21.0)
df[df.user == 372] = df[df.user == 372].fillna(30.0)
df[df.user == 544] = df[df.user == 544].fillna(30.0)
df[df.user == 406] = df[df.user == 406].fillna(30.0)
df[df.user == 403] = df[df.user == 403].fillna(30.0)
df[df.user == 375] = df[df.user == 375].fillna(31.0)

# add sections
df['section'] = np.where(df.cohort == 55, 'ds',
                             np.where(df.cohort == 34, 'ds',
                                     np.where(df.cohort == 30, 'ds',
                                             np.where(df.cohort == 28, 'staff',
                                                      np.where(df.cohort == 1, 'staff_1',
                                                               np.where(df.cohort == 8, 'staff','web_dev'))))))

df.head()

,page,user,cohort,ip,section
datetime,,,,,
2018-01-26 09:55:03,/,1,8.0,98.106.20.62,staff
2018-01-26 09:56:02,java-ii,1,8.0,98.106.20.62,staff
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,98.106.20.62,staff
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,98.106.20.62,staff
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,98.106.20.62,web_dev


In [10]:
df[df.cohort.isna()]

,page,user,cohort,ip,section
datetime,,,,,
2018-01-28 23:42:55,/,54,NaN,25.161.160.123,web_dev
2018-01-28 23:43:12,spring/extra-features,54,NaN,25.161.160.123,web_dev
2018-01-29 00:00:06,/,54,NaN,25.161.160.123,web_dev
2018-01-29 00:00:10,mkdocs/search_index.json,54,NaN,25.161.160.123,web_dev
2018-01-29 00:00:24,appendix/capstone-workbook,54,NaN,25.161.160.123,web_dev
...,...,...,...,...,...
2020-05-03 15:01:25,/,88,NaN,100.58.18.36,web_dev
2020-05-03 15:01:26,/,88,NaN,100.58.18.36,web_dev
2020-05-03 15:01:28,/,88,NaN,100.58.18.36,web_dev


In [ ]:
# make df of daily page views per ip

ip_daily_pages = df.page.resample('D').count()